Multiple hypothesis testing occurs when we repeatedly test models on a number of features, as the probability of obtaining one or more false discoveries increases with the number of tests. For example, in the field of genomics, scientists often want to test whether any of the thousands of genes have a significantly different activity in an outcome of interest.

In this blog post, we will cover few of the popular methods used to account for multiple hypothesis testing by adjusting model p-values:

1. False Positive Rate (FPR)
2. Family-Wise Error Rate (FWER)
3. False Discovery Rate (FDR)

 and explain when it makes sense to use them.

This document can be summarized in the following image:

<img src="images/pvalue_adjustments.png" width=90% height=90%>

# Create test data

We will create a simulated example to better understand how various manipulation of p-values can lead to different conclusions. To run this code, we need Python with `pandas`, `numpy`, `scipy` and `statsmodels` libraries installed.

For the purpose of this example, we start by creating a dataframe of 1000 features. 990 of which (99%) will have their values generated from a Normal distribution with mean = 0, called a Null model. (In a function `norm.rvs()` used below, mean is set using a `loc` argument.) The remaining 1% of the features will be generated from a Normal distribution mean = 3, called a Non-Null model. We will use these as representing interesting features that we would like to discover.

In [67]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from statsmodels.stats.multitest import multipletests

In [68]:
np.random.seed(42)

n_null = 9900
n_nonnull = 100

df = pd.DataFrame({
    'hypothesis': np.concatenate((
        ['null'] * n_null,
        ['non-null'] * n_nonnull,
    )),
    'feature': range(n_null + n_nonnull),
    'x': np.concatenate((
        norm.rvs(loc=0, scale=1, size=n_null),
        norm.rvs(loc=3, scale=1, size=n_nonnull),
    ))
})

For each of the 1000 features, p-value is a probability of observing the value at least as large, if we assume it was generated from a Null distribution.

P-values can be calculating from cumulative distribution. Cumulative distribution from `scipy.stats`, is named `norm.cdf()` and represents the probability of obtaining a value equal to or **less than** the one observed. Then to calculate the p-value we calculate `1 - norm.cdf()` to find the probability **greater than** the one observed:

In [69]:
df['p_value'] = 1 - norm.cdf(df['x'], loc = 0, scale = 1)
df

,hypothesis,feature,x,p_value
0,null,0,0.496714,0.309695
1,null,1,-0.138264,0.554984
2,null,2,0.647689,0.258593
3,null,3,1.523030,0.063876
4,null,4,-0.234153,0.592567
...,...,...,...,...
9995,non-null,9995,4.301102,0.000008
9996,non-null,9996,1.001655,0.158255
9997,non-null,9997,2.294683,0.010876
9998,non-null,9998,3.495766,0.000236


# False Positive Rate

The first concept is called a False Positive Rate, and is defined as a fraction of null hypothesis that we flag as "significant" (also called Type I errors). The p-values we calculated earlier can be interpreted as a false positive rate by their very definition: they are probabilities of obtaining a value at least as large as a specified value, when we sample a null distribution.

For illustrative purposes, we will apply a common (magical) p-value threshold of 0.05, but any threshold can be used:

In [70]:
df['is_raw_p_value_significant'] = df['p_value'] <= 0.05
df.groupby(['hypothesis', 'is_raw_p_value_significant']).size()

hypothesis  is_raw_p_value_significant
non-null    False                            8
            True                            92
null        False                         9407
            True                           493
dtype: int64

notice that out of our 9900 null hypotheses, 493 are flagged as "significant". Therefore, a False Positive Rate is: FPR =  493 / (493 + 9940) = 0.053. 

The main problem with FPR is that in a real scenario we do not a priori know which hypotheses are null and which are not. Then, the raw p-value on its own (False Positive Rate) can be of little use. In our case when the fraction of non-null features is very small, most of the features flagged as significant will be null, because there are many more of them. Specifically, out of 92 + 493 = 585 features flagged true ("positive"), only 92 are from our non-null distribution. That means that a majority or about 84% of reported significant features (493 / 585) are false positives!

So what can we do about this? There are two common methods of addressing this issue: instead of False Positive Rate, we can calculate Family-Wise Error Rate (FWER) or a False Discovery Rate (FPR). Each of these methods takes the set of raw, unadjusted, p-values as an input, and produces a new set of "adjusted p-values" as an output. These "adjusted p-values" represent estimates of *upper bounds* on FWER and FDR. They can be obtained from `multipletests()` function, which is part of the `statsmodels` Python library:

In [71]:
def adjust_pvalues(p_values, method):
   return multipletests(p_values, method = method)[1]

# Family-Wise Error Rate

Family-Wise Error Rate is a probability of falsly rejecting one or more null hypothesis, i.e. flagging true Null as Non-null. In other words, this is a probability of obtaining one or more false positives. 

When there is only one hypothesis being tested, this is equal to the raw p-value (false positive rate). However, the more hypotheses are tested, the more likely we are going to get one or more false positives. There are two popular ways to estimate FWER: Bonferroni and Holm procedures. Although neither Bonferroni or Holm procedures make any assumptions about the dependence of tests run on individual features, they will be overly conservative. For example, in the extreme case when all of the features are identical (same model repeated 10,000 times), no correction is needed. While in the other extreme, where no features are correlated, some type of correction is required.

## Bonferroni procedure

One of the most popular method for correcting for multiple hypothesis testing is a Bonferroni procedure. The reason this method is popular is because it is very easy to calculate, even by hand. This procedure multiplies each p-value by the total number of tests performed, or sets it to 1 if this multiplication would push it past 1.

In [72]:
df['p_value_bonf'] = adjust_pvalues(df['p_value'], 'bonferroni')
df.sort_values('p_value_bonf')

,hypothesis,feature,x,p_value,is_raw_p_value_significant,p_value_bonf
9907,non-null,9907,5.322609,5.114466e-08,True,0.000511
9942,non-null,9942,5.022174,2.554492e-07,True,0.002554
9943,non-null,9943,4.831177,6.786409e-07,True,0.006786
9941,non-null,9941,4.801528,7.872958e-07,True,0.007873
9976,non-null,9976,4.674271,1.475000e-06,True,0.014750
...,...,...,...,...,...,...
3336,null,3336,-1.365824,9.140029e-01,False,1.000000
3337,null,3337,-0.148969,5.592111e-01,False,1.000000
3338,null,3338,0.502784,3.075579e-01,False,1.000000
3331,null,3331,-1.545730,9.389151e-01,False,1.000000


## Holm procedure

Holm's procedure provides a correction that is more powerful than Bonferroni's procedure. The only difference is that the p-values are not all multiplied by the total number of tests (here, 10,000). Instead, each sorted p-value is multiplied progressively by a decreasing sequence 10,000, 9,999, 9,998, 9,997, ..., 3, 2, 1.

In [73]:
df['p_value_holm'] = adjust_pvalues(df['p_value'], 'holm')
df.sort_values('p_value_holm').head(10)

,hypothesis,feature,x,p_value,is_raw_p_value_significant,p_value_bonf,p_value_holm
9907,non-null,9907,5.322609,5.114466e-08,True,0.000511,0.000511
9942,non-null,9942,5.022174,2.554492e-07,True,0.002554,0.002554
9943,non-null,9943,4.831177,6.786409e-07,True,0.006786,0.006785
9941,non-null,9941,4.801528,7.872958e-07,True,0.007873,0.007871
9976,non-null,9976,4.674271,1.475000e-06,True,0.014750,0.014744
9964,non-null,9964,4.589147,2.225301e-06,True,0.022253,0.022242
9974,non-null,9974,4.515318,3.161090e-06,True,0.031611,0.031592
9990,non-null,9990,4.433625,4.633087e-06,True,0.046331,0.046298
9909,non-null,9909,4.414029,5.073215e-06,True,0.050732,0.050692
9916,non-null,9916,4.316007,7.943832e-06,True,0.079438,0.079367


We can verify this ourselves: the last 10th p-value on this output is multiplied by 9991: 7.943832e-06 * 9991 = 0.079367. Holm's correction is also the default method for adjusting p-values in `p.adjust()` function in R language.

If we again apply our p-value threshold of 0.05, let's take a look how these adjusted p-values affect our predictions:

In [74]:
df['is_p_value_holm_significant'] = df['p_value_holm'] <= 0.05
df.groupby(['hypothesis', 'is_p_value_holm_significant']).size()

hypothesis  is_p_value_holm_significant
non-null    False                            92
            True                              8
null        False                          9900
dtype: int64

These results are much different than when we applied the same threshold to the raw p-values! Here only 8 features are flagged as "significant", and all 8 are correct - they were generated from our Non-null distribution. This is because the probability of getting even one feature flagged incorrectly is only 0.05 (5%).

However, this approach has another downside: it failed to flag other 92 Non-null features as significant. While it was very stringent to make sure no null features slipped in, it  was able to find only 8% (8 out of 100) non-null features. This seems to be a different extreme than the False Positive Rate approach.

Is there a more middle ground? The answer is "yes", and that middle ground is False Discovery Rate.

# False Discovery Rate

What if we are OK with letting some false positives in, but capturing much more than few percent of true positives?

This can be done by controlling the False Discovery Rate (rather than FWER or FPR) at a specified threshold level, say 0.05. False Discovery Rate is defined a fraction of false positives among all features flagged as positive: FDR = FP / (FP + TP), where FP is the number of False Positives and TP is the number of True Positives.

There are several methods to control FDR and here we will describe how to use two popular ones: Benjamini-Hochberg and Benjamini-Yekutieli procedures. Both of these procedures are similiar although more involved than FWER procedures. They still rely on sorting the p-values, multiplying them with a specific number, and then using a cutoff criteria.

## Benjamini-Hochberg procedure

Benjamini-Hochberg (BH) procedure relies assumes that each of the tests are *independent*. Dependent tests occur, for example, if the features being testede are correlated with each other. Let's calculate the BH-adjusted p-values and compare it to our earlier result from FWER using Holm's correction:

In [75]:
df['p_value_bh'] = adjust_pvalues(df['p_value'], 'fdr_bh')
df[['hypothesis', 'feature', 'x', 'p_value', 'p_value_holm', 'p_value_bh']] \
    .sort_values('p_value_bh') \
    .head(10)

,hypothesis,feature,x,p_value,p_value_holm,p_value_bh
9907,non-null,9907,5.322609,5.114466e-08,0.000511,0.000511
9942,non-null,9942,5.022174,2.554492e-07,0.002554,0.001277
9941,non-null,9941,4.801528,7.872958e-07,0.007871,0.001968
9943,non-null,9943,4.831177,6.786409e-07,0.006785,0.001968
9976,non-null,9976,4.674271,1.475000e-06,0.014744,0.002950
9964,non-null,9964,4.589147,2.225301e-06,0.022242,0.003709
9974,non-null,9974,4.515318,3.161090e-06,0.031592,0.004516
9990,non-null,9990,4.433625,4.633087e-06,0.046298,0.005637
9909,non-null,9909,4.414029,5.073215e-06,0.050692,0.005637
9995,non-null,9995,4.301102,8.497538e-06,0.084890,0.007097


In [76]:
df['is_p_value_holm_significant'] = df['p_value_holm'] <= 0.05
df.groupby(['hypothesis', 'is_p_value_holm_significant']).size()

hypothesis  is_p_value_holm_significant
non-null    False                            92
            True                              8
null        False                          9900
dtype: int64

In [77]:
df['is_p_value_bh_significant'] = df['p_value_bh'] <= 0.05
df.groupby(['hypothesis', 'is_p_value_bh_significant']).size()

hypothesis  is_p_value_bh_significant
non-null    False                          67
            True                           33
null        False                        9898
            True                            2
dtype: int64

BH procedure now correctly flagged 33 out of 100 non-null features as significant - an improvement from the 8 with the Holm's correction. However, it also flagged 2 null features as significant. So, out of the 35 features flagged as significant, the fraction of incorrect features is: 2 / 33 = 0.06 so 6%.

Note that in this case we have 6% FDR rate, even though we aimed to controll it at 5%. FDR will be controlled at a 5% rate *on average*: sometimes it may be lower and sometimes it may be higher.

## Benjamini-Yekutieli procedure

Benjamini-Yekutieli (BY) procedure controls FDR regardless of whether tests are independent or not. Again, it is worth noting that all of these procedures try to establish *upper bounds* on FDR (or FWER), so they may be less or more conservative. Let's compare the BY procedure with a BH and Holm procedures above:

In [78]:
df['p_value_by'] = adjust_pvalues(df['p_value'], 'fdr_by')
df[['hypothesis', 'feature', 'x', 'p_value', 'p_value_holm', 'p_value_bh', 'p_value_by']] \
    .sort_values('p_value_by') \
    .head(10)

,hypothesis,feature,x,p_value,p_value_holm,p_value_bh,p_value_by
9907,non-null,9907,5.322609,5.114466e-08,0.000511,0.000511,0.005006
9942,non-null,9942,5.022174,2.554492e-07,0.002554,0.001277,0.012501
9943,non-null,9943,4.831177,6.786409e-07,0.006785,0.001968,0.019264
9941,non-null,9941,4.801528,7.872958e-07,0.007871,0.001968,0.019264
9976,non-null,9976,4.674271,1.475000e-06,0.014744,0.002950,0.028873
9964,non-null,9964,4.589147,2.225301e-06,0.022242,0.003709,0.036301
9974,non-null,9974,4.515318,3.161090e-06,0.031592,0.004516,0.044199
9990,non-null,9990,4.433625,4.633087e-06,0.046298,0.005637,0.055172
9909,non-null,9909,4.414029,5.073215e-06,0.050692,0.005637,0.055172
9916,non-null,9916,4.316007,7.943832e-06,0.079367,0.007097,0.069458


In [79]:
df['is_p_value_by_significant'] = df['p_value_by'] <= 0.05
df.groupby(['hypothesis', 'is_p_value_by_significant']).size()

hypothesis  is_p_value_by_significant
non-null    False                          93
            True                            7
null        False                        9900
dtype: int64

BY procedure is stricter in controlling FDR; in this case even more so than the Holm's procedure for controlling FWER, by flagging only 7 non-null features as significant. Although the main advantage of using it is when we know the data may contain a high number of correlated features.

# Summary

At the end, the choice of procedure is left to the user and depends on what the analysis is trying to do. Quoting Benjamini, Hochberg (Royal Stat. Soc. 1995):

> Often the control of the FWER is not quite needed. The control of the FWER is important when a conclusion from the various individual inferences is likely to be erroneous when at least one ofthem is.
> 
> This may be the case, for example, when several new treatments are competing against a standard, and a single treatment is chosen from the set of treatments which are declared significantly better than the standard.

In other cases, where we may be OK to have some false positives, FDR methods provide less stringent p-value adjustments and may be preferrable if we primarily want to increase the number of true positives that pass a certain p-value threshold.